In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report

# ===================================================================
# ARQUITECTURA: PASAR TODOS LOS CSV TAL CUAL
# ===================================================================
print("="*60)
print("DEEP LEARNING MULTI-INPUT: CSV TAL CUAL")
print("="*60)
print("\n💡 Este modelo recibe:")
print("   - Datos de application (fijos)")
print("   - Secuencia variable de créditos bureau")
print("   - Secuencia variable de pagos installments")
print("   - Secuencia variable de previous applications")

# ===================================================================
# 1. CARGAR DATOS
# ===================================================================
print("\n" + "="*60)
print("CARGANDO DATOS")
print("="*60)

app_train = pd.read_csv('../data/raw/application_tr.csv')
bureau = pd.read_csv('../data/raw/bureau.csv')
installments = pd.read_csv('../data/raw/installments_payments.csv')
previous = pd.read_csv('../data/raw/previous_application.csv')

print(f"\n✓ Application: {app_train.shape}")
print(f"✓ Bureau: {bureau.shape}")
print(f"✓ Installments: {installments.shape}")
print(f"✓ Previous: {previous.shape}")

# ===================================================================
# 2. PREPARAR FEATURES NUMÉRICAS
# ===================================================================
print("\n" + "="*60)
print("PREPARANDO FEATURES")
print("="*60)

# APPLICATION: Features fijas del cliente
app_numeric_cols = [
    'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
    'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
    'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
    'HOUR_APPR_PROCESS_START', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'
]
app_numeric_cols = [c for c in app_numeric_cols if c in app_train.columns]

# BUREAU: Features de cada crédito
bureau_numeric_cols = [
    'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
    'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
    'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
    'DAYS_CREDIT_UPDATE'
]
bureau_numeric_cols = [c for c in bureau_numeric_cols if c in bureau.columns]

# INSTALLMENTS: Features de cada pago
inst_numeric_cols = [
    'NUM_INSTALMENT_VERSION', 'NUM_INSTALMENT_NUMBER',
    'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
    'AMT_INSTALMENT', 'AMT_PAYMENT'
]
inst_numeric_cols = [c for c in inst_numeric_cols if c in installments.columns]

# PREVIOUS: Features de cada solicitud previa
prev_numeric_cols = [
    'AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT',
    'AMT_GOODS_PRICE', 'HOUR_APPR_PROCESS_START', 'DAYS_DECISION',
    'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION'
]
prev_numeric_cols = [c for c in prev_numeric_cols if c in previous.columns]

print(f"\n✓ Application features: {len(app_numeric_cols)}")
print(f"✓ Bureau features: {len(bureau_numeric_cols)}")
print(f"✓ Installments features: {len(inst_numeric_cols)}")
print(f"✓ Previous features: {len(prev_numeric_cols)}")

# Llenar NaN
app_train[app_numeric_cols] = app_train[app_numeric_cols].fillna(0)
bureau[bureau_numeric_cols] = bureau[bureau_numeric_cols].fillna(0)
installments[inst_numeric_cols] = installments[inst_numeric_cols].fillna(0)
previous[prev_numeric_cols] = previous[prev_numeric_cols].fillna(0)

# ===================================================================
# 3. CREAR GENERADOR DE DATOS (MANEJA SECUENCIAS VARIABLES)
# ===================================================================
print("\n" + "="*60)
print("CREANDO GENERADOR DE DATOS")
print("="*60)

class MultiInputDataGenerator(keras.utils.Sequence):
    """
    Generador que crea batches con secuencias de longitud variable
    """
    def __init__(self, client_ids, batch_size=32, shuffle=True):
        self.client_ids = client_ids
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.client_ids) / self.batch_size))

    def __getitem__(self, idx):
        batch_ids = self.client_ids[idx * self.batch_size:(idx + 1) * self.batch_size]
        return self._generate_batch(batch_ids)

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.client_ids))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            self.client_ids = self.client_ids[self.indexes]

    def _generate_batch(self, batch_ids):
        # Arrays para el batch
        app_batch = []
        bureau_batch = []
        inst_batch = []
        prev_batch = []
        targets = []

        max_bureau = 0
        max_inst = 0
        max_prev = 0

        # Primera pasada: encontrar longitudes máximas en este batch
        for client_id in batch_ids:
            bureau_client = bureau[bureau['SK_ID_CURR'] == client_id]
            inst_client = installments[installments['SK_ID_CURR'] == client_id]
            prev_client = previous[previous['SK_ID_CURR'] == client_id]

            max_bureau = max(max_bureau, len(bureau_client))
            max_inst = max(max_inst, len(inst_client))
            max_prev = max(max_prev, len(prev_client))

        # Segunda pasada: crear tensores con padding
        for client_id in batch_ids:
            # Application (fijo)
            app_data = app_train[app_train['SK_ID_CURR'] == client_id]
            app_features = app_data[app_numeric_cols].values[0]
            app_batch.append(app_features)

            # Target
            targets.append(app_data['TARGET'].values[0])

            # Bureau (variable - se hace padding)
            bureau_client = bureau[bureau['SK_ID_CURR'] == client_id][bureau_numeric_cols].values
            if len(bureau_client) == 0:
                bureau_client = np.zeros((1, len(bureau_numeric_cols)))
            bureau_padded = np.zeros((max_bureau, len(bureau_numeric_cols)))
            bureau_padded[:len(bureau_client)] = bureau_client
            bureau_batch.append(bureau_padded)

            # Installments (variable - se hace padding)
            inst_client = installments[installments['SK_ID_CURR'] == client_id][inst_numeric_cols].values
            if len(inst_client) == 0:
                inst_client = np.zeros((1, len(inst_numeric_cols)))
            # Limitar a max 100 pagos para no explotar memoria
            inst_client = inst_client[:100]
            inst_padded = np.zeros((min(max_inst, 100), len(inst_numeric_cols)))
            inst_padded[:len(inst_client)] = inst_client
            inst_batch.append(inst_padded)

            # Previous (variable - se hace padding)
            prev_client = previous[previous['SK_ID_CURR'] == client_id][prev_numeric_cols].values
            if len(prev_client) == 0:
                prev_client = np.zeros((1, len(prev_numeric_cols)))
            prev_padded = np.zeros((max_prev, len(prev_numeric_cols)))
            prev_padded[:len(prev_client)] = prev_client
            prev_batch.append(prev_padded)

        return (
            {
                'application': np.array(app_batch),
                'bureau': np.array(bureau_batch),
                'installments': np.array(inst_batch),
                'previous': np.array(prev_batch)
            },
            np.array(targets)
        )

print("✓ Generador creado")

# ===================================================================
# 4. CONSTRUIR MODELO MULTI-INPUT
# ===================================================================
print("\n" + "="*60)
print("CONSTRUYENDO ARQUITECTURA NEURAL")
print("="*60)

# INPUT 1: Application (datos fijos del cliente)
app_input = layers.Input(shape=(len(app_numeric_cols),), name='application')
app_dense = layers.Dense(64, activation='relu')(app_input)
app_dense = layers.BatchNormalization()(app_dense)
app_dense = layers.Dropout(0.3)(app_dense)
app_dense = layers.Dense(32, activation='relu')(app_dense)

# INPUT 2: Bureau (secuencia variable de créditos)
bureau_input = layers.Input(shape=(None, len(bureau_numeric_cols)), name='bureau')
bureau_mask = layers.Masking(mask_value=0.0)(bureau_input)
bureau_lstm = layers.LSTM(64, return_sequences=False)(bureau_mask)
bureau_lstm = layers.Dropout(0.3)(bureau_lstm)

# INPUT 3: Installments (secuencia variable de pagos)
inst_input = layers.Input(shape=(None, len(inst_numeric_cols)), name='installments')
inst_mask = layers.Masking(mask_value=0.0)(inst_input)
inst_lstm = layers.LSTM(64, return_sequences=False)(inst_mask)
inst_lstm = layers.Dropout(0.3)(inst_lstm)

# INPUT 4: Previous (secuencia variable de solicitudes previas)
prev_input = layers.Input(shape=(None, len(prev_numeric_cols)), name='previous')
prev_mask = layers.Masking(mask_value=0.0)(prev_input)
prev_lstm = layers.LSTM(32, return_sequences=False)(prev_mask)
prev_lstm = layers.Dropout(0.3)(prev_lstm)

# COMBINAR todas las representaciones
combined = layers.concatenate([app_dense, bureau_lstm, inst_lstm, prev_lstm])
combined = layers.Dense(128, activation='relu')(combined)
combined = layers.BatchNormalization()(combined)
combined = layers.Dropout(0.4)(combined)
combined = layers.Dense(64, activation='relu')(combined)
combined = layers.Dropout(0.3)(combined)

# OUTPUT: Probabilidad de default
output = layers.Dense(1, activation='sigmoid', name='output')(combined)

# Crear modelo
model = keras.Model(
    inputs=[app_input, bureau_input, inst_input, prev_input],
    outputs=output
)

# Compilar con class weight para balancear
class_weight = {0: 1.0, 1: 10.0}  # Más peso a clase minoritaria

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['AUC', 'accuracy']
)

print("\n✓ Modelo construido")
print("\n📊 Arquitectura:")
model.summary()

# ===================================================================
# 5. ENTRENAR
# ===================================================================
print("\n" + "="*60)
print("ENTRENAMIENTO")
print("="*60)

# Split clientes
unique_clients = app_train['SK_ID_CURR'].unique()
train_clients, val_clients = train_test_split(
    unique_clients, test_size=0.2, random_state=42
)

print(f"\nClientes train: {len(train_clients)}")
print(f"Clientes val: {len(val_clients)}")

# Crear generadores
train_gen = MultiInputDataGenerator(train_clients, batch_size=64, shuffle=True)
val_gen = MultiInputDataGenerator(val_clients, batch_size=64, shuffle=False)

# Callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_auc',
        patience=5,
        mode='max',
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6
    )
]

print("\n🚀 Iniciando entrenamiento...")
print("   (Esto puede tomar varios minutos...)\n")

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=callbacks,
    class_weight=class_weight,
    verbose=1
)

# ===================================================================
# 6. EVALUAR
# ===================================================================
print("\n" + "="*60)
print("EVALUACIÓN")
print("="*60)

# Predecir en validación
print("\nGenerando predicciones...")
y_pred_proba = []
y_true = []

for i in range(len(val_gen)):
    batch_x, batch_y = val_gen[i]
    preds = model.predict(batch_x, verbose=0)
    y_pred_proba.extend(preds.flatten())
    y_true.extend(batch_y)

y_pred_proba = np.array(y_pred_proba)
y_true = np.array(y_true)

# Métricas
auc_score = roc_auc_score(y_true, y_pred_proba)
y_pred = (y_pred_proba >= 0.5).astype(int)

print(f"\n🎯 AUC Score: {auc_score:.4f}")
print(f"\n{classification_report(y_true, y_pred)}")

# ===================================================================
# 7. FUNCIÓN DE PREDICCIÓN
# ===================================================================
def predict_client_deep_learning(client_id):
    """
    Predice usando el modelo de deep learning multi-input
    """
    print(f"\n{'='*60}")
    print(f"PREDICCIÓN PARA CLIENTE {client_id}")
    print(f"{'='*60}\n")

    # Preparar datos
    app_data = app_train[app_train['SK_ID_CURR'] == client_id]
    if len(app_data) == 0:
        print("❌ Cliente no encontrado")
        return None

    app_features = app_data[app_numeric_cols].values

    bureau_client = bureau[bureau['SK_ID_CURR'] == client_id][bureau_numeric_cols].values
    if len(bureau_client) == 0:
        bureau_client = np.zeros((1, len(bureau_numeric_cols)))

    inst_client = installments[installments['SK_ID_CURR'] == client_id][inst_numeric_cols].values
    if len(inst_client) == 0:
        inst_client = np.zeros((1, len(inst_numeric_cols)))
    inst_client = inst_client[:100]  # Limitar

    prev_client = previous[previous['SK_ID_CURR'] == client_id][prev_numeric_cols].values
    if len(prev_client) == 0:
        prev_client = np.zeros((1, len(prev_numeric_cols)))

    print(f"✓ Datos encontrados:")
    print(f"   - Application features: {len(app_features[0])}")
    print(f"   - Bureau records: {len(bureau_client)}")
    print(f"   - Installment records: {len(inst_client)}")
    print(f"   - Previous records: {len(prev_client)}")

    # Predecir
    pred = model.predict({
        'application': np.expand_dims(app_features, 0),
        'bureau': np.expand_dims(bureau_client, 0),
        'installments': np.expand_dims(inst_client, 0),
        'previous': np.expand_dims(prev_client, 0)
    }, verbose=0)[0][0]

    print(f"\n🎯 Probabilidad de default: {pred:.1%}")

    if pred < 0.3:
        decision = "✅ BAJO RIESGO"
    elif pred < 0.6:
        decision = "⚠️ RIESGO MODERADO"
    else:
        decision = "❌ ALTO RIESGO"

    print(f"   Decisión: {decision}")

    return pred

# ===================================================================
# 8. EJEMPLO
# ===================================================================
print("\n" + "="*60)
print("EJEMPLO DE PREDICCIÓN")
print("="*60)

example_client = val_clients[0]
predict_client_deep_learning(example_client)

print("\n" + "="*60)
print("✅ COMPLETADO")
print("="*60)
print("\n💡 Este modelo:")
print("   ✅ Usa TODOS los datos de TODOS los CSV tal cual")
print("   ✅ No necesita agregaciones manuales")
print("   ✅ Aprende automáticamente qué es importante")
print("   ✅ Maneja secuencias de longitud variable")
print("   ❌ Menos interpretable que XGBoost")
print("   ❌ Más lento de entrenar")
print("   ❌ Requiere más memoria")

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
app_train = pd.read_csv('../data/raw/application_tr.csv')
app_test = pd.read_csv('../data/raw/application_ts.csv')
bureau = pd.read_csv('../data/raw/bureau.csv')
bureau_balance = pd.read_csv('../data/raw/bureau_balance.csv')
previous = pd.read_csv('../data/raw/previous_application.csv')
pos_cash = pd.read_csv('../data/raw/POS_CASH_balance.csv')
credit_card = pd.read_csv('../data/raw/credit_card_balance.csv')
installments = pd.read_csv('../data/raw/installments_payments.csv')

df_train = app_train.copy()

df_train = df_train.merge(
    bureau,
    on="SK_ID_CURR",
    how="left")

df_train = df_train.merge(
    bureau_balance,
    on="SK_ID_BUREAU",
    how="left")

df_train = df_train.merge(
    previous,
    on="SK_ID_CURR",
    how="left")

df_train = df_train.merge(
    pos_cash,
    on="SK_ID_PREV",
    how="left")

df_train = df_train.merge(
    credit_card,
    on="SK_ID_PREV",
    how="left")

df_train = df_train.merge(
    installments,
    on="SK_ID_PREV",
    how="left")

df_test = app_test.copy()

df_test = df_test.merge(bureau, on="SK_ID_CURR", how="left")
df_test = df_test.merge(bureau_balance, on="SK_ID_BUREAU", how="left")
df_test = df_test.merge(previous, on="SK_ID_CURR", how="left")
df_test = df_test.merge(pos_cash, on="SK_ID_PREV", how="left")
df_test = df_test.merge(credit_card, on="SK_ID_PREV", how="left")
df_test = df_test.merge(installments, on="SK_ID_PREV", how="left")


